# **AI bootcamp Final Project**

# **AI Pilot Assistant**  

The AI Pilot Assistant provides pilots with real-time data access in an easy and efficient manner. It also assists them in emergency procedures, ensuring quick decision-making and enhanced flight safety.

### **Install Libraries**

In [ ]:
!pip install langchain streamlit requests metar langchain-community langchain_huggingface pypdf langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/8

### **Import Api keys**

In [ ]:
import os
from langchain_groq import ChatGroq

#impot CheckWV Api
from google.colab import userdata
key = userdata.get('checkwv')
os.environ["CHECKWX_API_KEY"] = key
checkwv_api = os.environ["CHECKWX_API_KEY"]

#Import the groq Api
from google.colab import userdata
key1 = userdata.get('groq')
os.environ["GROQ_API_KEY"] = key1

from google.colab import userdata
key2 = userdata.get('aviaton_stack')
os.environ["AVIATION_STACK_API_KEY"] = key2
aviation_stack_key = os.environ["AVIATION_STACK_API_KEY"]


llm = ChatGroq(
    temperature=0.1,
    model_name="mixtral-8x7b-32768",
    api_key=key1,
)

### **Defined Customize Tools Functions**  

In [ ]:
from langchain.tools import tool
import requests
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.tools import tool

loader = PyPDFLoader("/content/drive/MyDrive/Emergency_Procedure data.pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = InMemoryVectorStore.from_documents(splits, embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Emergency Procedure Tool
@tool
def emergency_procedures_tool(query: str) -> str:
    """
    Retrieves emergency procedures for aviation scenarios.
    """
    # Use FAISS to retrieve relevant procedures
    docs = vectorstore.similarity_search(query, k=3)
    return "\n".join([doc.page_content for doc in docs])


# METAR Tool
@tool
def metar_tool(icao_code : str) -> str:
  """
  Fetches METAR data for a given airport.
  """
  url = f"https://api.checkwx.com/metar/{icao_code}"
  headers = {"X-API-Key": checkwv_api}
  response = requests.get(url, headers=headers)
  data = response.json()
  return data["data"][0] if data.get("results") > 0 else "No METAR data found."


# Flight Data Tool
@tool
def flight_data_tool(icao24: str) -> str:
    """
    Fetches live flight data for a given aircraft.
    """
    url = f"https://opensky-network.org/api/states/all?icao24={icao24}"
    response = requests.get(url)
    data = response.json()
    if data.get("states"):
        flight_data = data["states"][0]
        return f"""
        Callsign: {flight_data[1].strip()}
        Altitude: {flight_data[7]} meters
        Speed: {flight_data[9]} m/s
        Heading: {flight_data[10]}°
        Latitude: {flight_data[6]}
        Longitude: {flight_data[5]}
        """
    return "No flight data found."


# Weather Forecast Tool
@tool
def weather_forecast_tool(city_name: str) -> str:
    """
    Fetches weather forecast for a given city.
    """
    url = "https://geocoding-api.open-meteo.com/v1/search"
    params = {"name": city_name, "count": 1, "format": "json"}
    response = requests.get(url, params=params)
    data = response.json()
    if data.get("results"):
        latitude, longitude = data["results"][0]["latitude"], data["results"][0]["longitude"]
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "hourly": "temperature_2m,weathercode",
            "forecast_days": 1,
        }
        response = requests.get(url, params=params)
        data = response.json()
        if "hourly" in data:
            forecast = []
            for i in range(len(data["hourly"]["time"])):
                forecast.append(
                    f"Time: {data['hourly']['time'][i]}, "
                    f"Temperature: {data['hourly']['temperature_2m'][i]}°C, "
                    f"Weather Code: {data['hourly']['weathercode'][i]}"
                )
            return "\n".join(forecast)
    return "No weather forecast data found."

# Flight Status Tool (Using Aviation Stack API)
@tool
def flight_status_tool(flight_number: str) -> str:
    """
    Get real-time flight status with delay and gate information.
    """
    try:
        url = "http://api.aviationstack.com/v1/flights"
        params = {"access_key": aviation_stack_key, "flight_iata": flight_number}
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()
        if data.get("data"):
            flight = data["data"][0]
            return f"""✈️ Flight {flight['flight']['iata']} ({flight['airline']['name']}):
Departure: {flight['departure']['airport']} at {flight['departure']['estimated']}
Arrival: {flight['arrival']['airport']} at {flight['arrival']['estimated']}
Status: {flight['flight_status']}
Delay: {flight['departure']['delay']} minutes"""
        return "Flight not found."
    except Exception as e:
        return f"⚠️ Error retrieving flight status: {str(e)}"

We define the tools used in our LangChain agent:  

**Tool 1**: **Emergency Procedure Tool** – Used to compare the user's query with pre-existing emergency procedure data to find similarities.  

**Tool 2**: **METAR Tool** – Fetches real-time weather conditions for airports.  

**Tool 3**: **Flight Data Tool** – Retrieves real-time data for a given aircraft, including altitude, speed, and other parameters.  

**Tool 4**: **Weather Forecast Tool** – Provides real-time weather data for a specified location.  

In [ ]:
from langchain.agents import initialize_agent, Tool

# Define the tools
tools = [
    Tool(
        name="Emergency Procedures",
        func=emergency_procedures_tool,
        description="Useful for retrieving emergency procedures for aviation scenarios.",
    ),
    Tool(
        name="METAR Data",
        func=metar_tool,
        description="Useful for fetching METAR data for airports.",
    ),
    Tool(
        name="Flight Data",
        func=flight_data_tool,
        description="Useful for retrieving live flight data.",
    ),
    Tool(
        name="Weather Forecast",
        func=weather_forecast_tool,
        description="Useful for fetching weather forecasts for destinations.",
    ),
    Tool(
        name="Flight Status",
        func=flight_status_tool,
        description="Useful for checking real-time flight status and delays.",
    ),
]

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Initialize memory for the chatbot
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

#prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an AI Pilot Assistant specializing in aviation support. Your capabilities include:

- Retrieving emergency procedures from official flight manuals
- Providing real-time METAR/TAF data for airports
- Accessing live flight tracking information
- Analyzing weather patterns and forecasts
- Checking real-time flight status and delays
- Calculating fuel requirements and flight plans

Follow these strict guidelines:
1. Use only aviation terminology (ICAO codes, METAR abbreviations, etc.)
2. For emergency situations, prioritize official procedures
3. Always verify data sources before presenting information
4. Redirect non-aviation queries to relevant aviation topics
5. Maintain professional pilot communication standards"""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)
# Modify the agent initialization
agent = initialize_agent(
    tools,
    llm,
    agent="conversational-react-description",
    memory=memory,
    verbose=True,
    prompt=prompt,
    max_iterations=3,
    early_stopping_method="generate",
    handle_parsing_errors=True,
    agent_kwargs={
        "prefix": "REMEMBER: You are a pilot assistant, not a general AI. "
                  "If asked non-aviation questions, respond with: "
                  "'I specialize in aviation support. How can I assist with your flight operations?'"
    }
)

### **Test Example**

In [ ]:
qurey = input("Ask me anything about aviation...")
agent.run(qurey)

### **Deployement of the Application on Hugging Face**
[Ai Pilot Assistant](https://huggingface.co/spaces/Aniq-63/AI_Pilot_Assistant)
